## Aufgaben 3

### (a) Datensatz einlesen
 Schreiben Sie Python-Code, mit dem Sie Ihren Datensatz einlesen können. Ihre erste Aufgabe ist es, die Daten in ein entsprechendes Datenformat zu transformieren, um es später vom Dashboard zugreifbar zu machen. Sie können die Daten entweder online zugreifen oder aber die Daten lokal auf Ihrem Rechner speichern und dort dann einlesen. Sind die Daten zu gross, dann lesen Sie nur einen Teil der Daten ein.



In [34]:
import pandas as pd

import plotly.graph_objects as go


from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px

from matplotlib import pyplot as plt
import matplotlib as mpl

In [35]:
#import data
df = pd.read_csv('Dataset_videogames sales.csv', sep=';')

In [36]:
#have a look at the data
df.head()

,Rank,Name,Platform,Platform Company,type of console,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,Nintendo,TV,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,Nintendo,TV,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,Nintendo,TV,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,Nintendo,TV,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,Nintendo,Portable,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [37]:
#have a look at the data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Rank              16598 non-null  int64  
 1   Name              16598 non-null  object 
 2   Platform          16572 non-null  object 
 3   Platform Company  16572 non-null  object 
 4   type of console   16572 non-null  object 
 5   Year              16301 non-null  float64
 6   Genre             16572 non-null  object 
 7   Publisher         16514 non-null  object 
 8   NA_Sales          16572 non-null  float64
 9   EU_Sales          16572 non-null  float64
 10  JP_Sales          16572 non-null  float64
 11  Other_Sales       16572 non-null  float64
 12  Global_Sales      16572 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 1.6+ MB


In [38]:
#transform column title to short name
df.columns = df.columns.str.replace('Global_Sales', 'Global')
df.columns = df.columns.str.replace('NA_Sales', 'North America')
df.columns = df.columns.str.replace('EU_Sales', 'Europe')
df.columns = df.columns.str.replace('JP_Sales', 'Japan')
df.columns = df.columns.str.replace('Other_Sales', 'Others')

# all values in column "Platform" to strings (for sorting filter and so on):
df['Platform'] = df['Platform'].map(str)

### (b) Filterfunktion

Implementieren Sie schon eine Filterfunktion, mit der man die Daten auch nur teilweise bezüglich eines oder mehrerer Attribute/Eigenschaften nutzen kann. (4 P.) Für welche Filteroptionen haben Sie sich entschieden? (1 P.)


In [39]:
# Filter Dropdown "Platform"

app = Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(id='dropdown_platform',
                 options=sorted([{'label': i, 'value': i} for i in df['Platform'].unique()], key = lambda x: x['label']),
                 style= {'width':'40%'}
                 ),
    html.Div(id='output-container')
])

@app.callback(
    Output('output-container', 'children'),
    Input('dropdown_platform', 'value')
)
def update_output(value):
    return f'You have selected {value}'


if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit


## Aufgabe 4 – Diagramme und Visualisierungen mit Plotly


### (a) Diagramm
Erzeugen Sie aus Ihrem Datensatz aus Aufgabe 3 ein Diagramm mit Plotly, das Sie in Ihrem Dashboard verwenden möchten. (5 P.) Interpretieren Sie das Diagramm, indem Sie Bezug auf die dargestellten Daten nehmen. (5 P.) Erzeugen Sie einen Screenshot des Streudiagrammes und fügen Sie diesen in das Lösungs-pdf ein.


In [40]:
def stacked_bar_chart_plotly(main_filter, sales_filter):
    # extract and copy date from df
    df_bar = df[[main_filter,'North America', 'Europe', 'Japan', 'Others', 'Global']]
    df_bar = df_bar[df_bar['Global'] > sales_filter]

    # group by filter and sort by global sales
    df_bar_grouped = df_bar.groupby([main_filter]).sum()
    df_bar_grouped = df_bar_grouped.sort_values(by=['Global'], ascending=False)

    #main filter as column
    df_bar_grouped.reset_index(inplace=True)
    df_bar_grouped = df_bar_grouped.rename(columns = {'index':main_filter})


    # Global Sales not shown
    df_bar_grouped = df_bar_grouped [[main_filter, 'North America', 'Europe', 'Japan', 'Others']]

    fig = px.bar(df_bar_grouped, x=main_filter, y=['North America', 'Europe', 'Japan', 'Others'], color_discrete_sequence= ['#1a889d', '#4da3b3', '#80bdc9', '#b3d7de'])
    fig.update_layout(plot_bgcolor='white',paper_bgcolor='white')
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', title = 'number of sales (in million)')

    fig.update_layout(legend=dict(
    title = '  Area',
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99
))
    fig.show()

stacked_bar_chart_plotly('Platform', 10)

### andere Diagramme

In [43]:
# line diagram

def line_diagram(main_filter, sales_filter):
    df_l = df[df['Global'] > sales_filter]
    df_l = df_l.groupby(['Year', main_filter], as_index=False)['Global'].sum()

    line_fig = px.line(df_l, x='Year', y='Global', color=main_filter, color_discrete_sequence= ['#015666', '#1a889d', '#4da3b3', '#80bdc9', '#b3d7de', '#cce5e9',  '#2b6b51', '#317a5c','#378a68','#50a381', '#77b89d', '#9eccb9' ])
    line_fig.update_layout(plot_bgcolor='white',paper_bgcolor='white')
    line_fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    line_fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
    line_fig.show()

line_diagram('Platform', 0)

In [44]:
# table1

def table1():
    import plotly.graph_objects as go

    df_ranking = df[['Rank', 'Name', 'Platform', 'Genre', 'Global']]

    fig = go.Figure(data=[go.Table(
        header=dict(values=list(df_ranking.columns),
                    fill_color='#80bdc9',
                    line_color='darkgray',
                    align='left'),
        cells=dict(values=[df_ranking.Rank, df_ranking.Name, df_ranking.Platform, df_ranking.Genre, df_ranking.Global],
                   fill_color='white',
                   line_color='darkgray',
                   align='left',
                   font_color='black'))
    ])

    fig.show()

table1()

In [45]:
# gauge chart

def gauge_chart(main_filter, sales_filter):
    regions = 'North America', 'Europe', 'Japan', 'Others'
    for region in regions:
        df_gauge = df[[main_filter,'North America', 'Europe', 'Japan', 'Others', 'Global']]
        df_gauge = df_gauge[df_gauge['Global'] > sales_filter]
        marktanteil_NA_Sales = round(df_gauge[region].sum() / df_gauge['Global'].sum()*100, 1)

        #Gauge Chart
        fig = go.Figure(go.Indicator(
            domain = {'x': [0, 1], 'y': [0, 1]},
            value = marktanteil_NA_Sales,
            number=  {'suffix': '%' },
            mode = 'gauge+number',
            title = {'text': region, 'font': { 'size': 42 }},
            gauge = {'axis': {'range': [None, 50]},
                     'bar': {'color': '#378a68'},
                     'steps' : [
                         {'range': [0, 50], 'color': '#cce5e9'},
                     ],
                         #{'range': [50/3, 50/3*2], 'color': "blue"},
                         #{'range': [50/3*2, 50], 'color': "green"}],
                     }))        # 'threshold' : {'line': {'color': "red", 'width': 4},'thickness': 0.75, 'value': 0.09}

        fig.show()

gauge_chart('Platform', 0)


### (b) Variabel Varianten
In diesem Aufgabenteil sollen Sie das Diagramm aus (a) so erweitern, dass es Variationen in den visuellen Variablen erlaubt, also etwa eine andere Farbe, andere Formen, ein anderes Layout oder aber textuelle Ergänzungen. Erzeugen Sie einen Screenshot des Diagrammes und fügen Sie diesen in das Lösungs-pdf ein. (5 P.)

In [46]:
def stacked_bar_chart_plotly(main_filter, sales_filter, color_choice):
    if color_choice == 'green':
        colortheme = ['#317a5c','#378a68','#50a381', '#77b89d']
    else:
        colortheme = ['#1a889d', '#4da3b3', '#80bdc9', '#b3d7de']
    # extract and copy date from df
    df_bar = df[[main_filter,'North America', 'Europe', 'Japan', 'Others', 'Global']]
    df_bar = df_bar[df_bar['Global'] > sales_filter]

    # group by filter and sort by global sales
    df_bar_grouped = df_bar.groupby([main_filter]).sum()
    df_bar_grouped = df_bar_grouped.sort_values(by=['Global'], ascending=False)

    #main filter as column
    df_bar_grouped.reset_index(inplace=True)
    df_bar_grouped = df_bar_grouped.rename(columns = {'index':main_filter})


    # Global Sales not shown
    df_bar_grouped = df_bar_grouped [[main_filter, 'North America', 'Europe', 'Japan', 'Others']]

    fig = px.bar(df_bar_grouped, x=main_filter, y=['North America', 'Europe', 'Japan', 'Others'], color_discrete_sequence= colortheme)
    fig.update_layout(plot_bgcolor='white',paper_bgcolor='white')
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', title = 'number of sales (in million)')

    fig.update_layout(legend=dict(
    title = '  Area',
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99
))
    fig.show()

stacked_bar_chart_plotly('Genre', 10, 'green')